In [1]:
import requests
import csv
from bs4 import BeautifulSoup as bs
from random import randint
from time import sleep
from urllib.parse import urljoin
import os
import snowflake.connector
from datetime import datetime
import uuid
import re
import pandas as pd
import time
from word2number import w2n  # Library to convert numbers to words

In [2]:
URL = 'https://www.espncricinfo.com/records/trophy/indian-premier-league-117'
sleep_delay = 5
#https://rforotb-ka22072.snowflakecomputing.com
# Snowflake connection parameters
snowflake_account = 'rforotb-ka22072'
snowflake_warehouse= 'COMPUTE_WH'
snowflake_database = 'MYCRICKET'
snowflake_schema = 'DATA1'
snowflake_user = 'comicsvibe'
snowflake_password = 'Comicsvibe123!'
snowflake_stage = 'MYSTAGE1'

# Establish Snowflake connection
conn = snowflake.connector.connect(
    account=snowflake_account,
    warehouse=snowflake_warehouse,
    database=snowflake_database,
    schema=snowflake_schema,
    user=snowflake_user,
    password=snowflake_password,
    stage=snowflake_stage
)

In [3]:
def remove_extra_chars(text):
    # Remove extra characters like double quotes from the text
    return text.replace('"', '')

def sanitize_filename(filename):
    # Replace any dashes (-) with underscores (_) in the filename
    filename = filename.replace('-', '_')

    # Add 'S_' prefix if the filename starts with a number
    if filename[0].isdigit():
        filename = f"S_{filename}"

    return filename

def generate_unique_filename(filename):
    # Generate a unique filename to avoid overwriting existing files
    if not os.path.exists(filename):
        return filename
    base, ext = os.path.splitext(filename)
    count = 1
    while os.path.exists(f"{base}_{count}{ext}"):
        count += 1
    return f"{base}_{count}{ext}"

def sanitize_column_name(col):
    # Replace '%' with 'Percentage', '/' with 'Slash', and '+' with 'Plus'
    col = col.replace('%', 'Percentage').replace('/', 'Slash').replace('+', 'Plus')

    # Convert column names with numbers to English spelling
    words = col.split()
    new_words = []
    for word in words:
        try:
            num = w2n.word_to_num(word)
            new_words.append(str(num))
        except ValueError:
            new_words.append(word)
    col = ' '.join(new_words)

    return col

def handle_duplicate_columns(data):
    # Rename columns that have the same name by appending '_1', '_2', etc.
    header = data[0]
    header_count = {col: 1 for col in header}
    for i in range(1, len(header)):
        col = header[i]
        if header.count(col) > 1:
            data[0][i] = f"{col}_{header_count[col]}"
            header_count[col] += 1
    return data

def remove_empty_columns(data):
    # Remove any empty columns from the data
    return [[col for col in row if col] for row in data]

req = requests.get(URL)
soup = bs(req.text, 'html.parser')

links = soup.find_all('a', href=True)

for link in links:
    absolute_url = urljoin(URL, link['href'])
    req = requests.get(absolute_url)
    print(f"Visiting page: {absolute_url}...")  # Print statement with the URL being visited
    soup = bs(req.text, 'html.parser')

    table = soup.find('table')
    if table:
        data = []
        rows = table.find_all('tr')
        for row in rows:
            data.append([remove_extra_chars(cell.get_text(strip=True)) for cell in row.find_all('td')])

        if data:
            # Handle duplicate column names by appending '_1', '_2', etc.
            data = handle_duplicate_columns(data)

            # Remove empty columns from the data
            data = remove_empty_columns(data)

            # Create a unique CSV filename based on the page URL
            csv_filename = f"{sanitize_filename(os.path.basename(absolute_url))}.csv"
            csv_filename = generate_unique_filename(csv_filename)

            # Generate 'uuid' and 'LAST_UPDATE' values in real-time during scraping
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            data_with_new_columns = [['uuid', 'LAST_UPDATE'] + [sanitize_column_name(col) for col in data[0]]] + [[str(uuid.uuid4()), current_time] + [col for col in row] for row in data[1:]]

            # Save data to the CSV file
            with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(data_with_new_columns)

            # Upload the CSV file to Snowflake stage and replace the existing file
            cursor = conn.cursor()
            remove_command = f"REMOVE @{snowflake_stage}/{csv_filename}"
            cursor.execute(remove_command)
            put_command = f"PUT file://{csv_filename} @{snowflake_stage}/{csv_filename}"
            cursor.execute(put_command)
            cursor.close()

            print(f"Table data scraped and saved to '{csv_filename}' and uploaded to Snowflake stage.")
        else:
            print("No table data found on the page.")

    sleep(sleep_delay)

print("All links visited.")

# Close the Snowflake connection
conn.close()

Visiting page: https://www.espncricinfo.com/...
Visiting page: https://www.espncricinfo.com/live-cricket-score...
Visiting page: https://www.espncricinfo.com/ci/content/match/fixtures_futures.html...
Visiting page: https://www.espncricinfo.com/team...
Visiting page: https://www.espncricinfo.com/cricket-news...
Visiting page: https://www.espncricinfo.com/cricket-features...
Visiting page: https://www.espncricinfo.com/cricket-videos/...
Visiting page: https://www.espncricinfo.com/records...
Visiting page: https://www.espncricinfo.com/records...
Visiting page: https://www.espncricinfo.com/records...
Visiting page: https://www.espncricinfo.com/records/year/2023-2023...
Visiting page: https://www.espncricinfo.com/ask...
Visiting page: https://stats.espncricinfo.com/ci/engine/stats/index.html...
Table data scraped and saved to 'index.html.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/trophy/world-cup-12...
Visiting page: https://www.espncricinfo.co

Visiting page: https://www.espncricinfo.com/records/most-sixes-in-an-innings-283019...
Table data scraped and saved to 'most_sixes_in_an_innings_283019.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-fours-in-an-innings-283015...
Table data scraped and saved to 'most_fours_in_an_innings_283015.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-sixes-in-a-match-283012...
Table data scraped and saved to 'most_sixes_in_a_match_283012.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-fours-in-a-match-283008...
Table data scraped and saved to 'most_fours_in_a_match_283008.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-runs-from-fours-and-sixes-in-an-innings-283004...
Table data scraped and saved to 'most_runs_from_fours_and_sixes_in_an_innings_283004.csv' and uploaded to Snowflake stage.
Visiting page: https:/

Visiting page: https://www.espncricinfo.com/records/most-runs-on-a-single-ground-284231...
Table data scraped and saved to 'most_runs_on_a_single_ground_284231.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-runs-off-one-over-284226...
Table data scraped and saved to 'most_runs_off_one_over_284226.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-runs-in-an-innings-by-a-captain-284218...
Table data scraped and saved to 'most_runs_in_an_innings_by_a_captain_284218.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-runs-in-an-innings-by-a-wicketkeeper-284209...
Table data scraped and saved to 'most_runs_in_an_innings_by_a_wicketkeeper_284209.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/highest-career-batting-average-282912...
Table data scraped and saved to 'highest_career_batting_average_282912.csv' and uploa

Visiting page: https://www.espncricinfo.com/records/best-figures-in-a-innings-by-a-captain-283935...
Table data scraped and saved to 'best_figures_in_a_innings_by_a_captain_283935.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/best-figures-in-a-innings-when-on-the-losing-side-283926...
Table data scraped and saved to 'best_figures_in_a_innings_when_on_the_losing_side_283926.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/best-career-bowling-average-283258...
Table data scraped and saved to 'best_career_bowling_average_283258.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/best-career-economy-rate-283267...
Table data scraped and saved to 'best_career_economy_rate_283267.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/best-career-strike-rate-283276...
Table data scraped and saved to 'best_career_strike_rate_283276.csv' a

Visiting page: https://www.espncricinfo.com/records/most-catches-in-an-innings-283353...
Table data scraped and saved to 'most_catches_in_an_innings_283353.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-stumpings-in-career-283338...
Table data scraped and saved to 'most_stumpings_in_career_283338.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-stumpings-in-an-innings-283327...
Table data scraped and saved to 'most_stumpings_in_an_innings_283327.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-byes-conceded-in-an-innings-283313...
Table data scraped and saved to 'most_byes_conceded_in_an_innings_283313.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/format/twenty20-internationals-3/category/fielding-records-6...
Visiting page: https://www.espncricinfo.com/records/most-catches-in-career-283650...
Table data 

Visiting page: https://www.espncricinfo.com/records/related-twenty20-international-players-312634...
Table data scraped and saved to 'related_twenty20_international_players_312634.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-matches-as-captain-283428...
Table data scraped and saved to 'most_matches_as_captain_283428.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/most-consecutive-matches-as-captain-of-a-team-1223146...
Table data scraped and saved to 'most_consecutive_matches_as_captain_of_a_team_1223146.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/winning-all-tosses-in-a-series-1223147...
Table data scraped and saved to 'winning_all_tosses_in_a_series_1223147.csv' and uploaded to Snowflake stage.
Visiting page: https://www.espncricinfo.com/records/youngest-captains-283414...
Table data scraped and saved to 'youngest_captains_283414.csv' and uploaded 